In [1]:
# 下载文件
import wget
import os
#os.chdir(r"E:\jupyter_project\用 Python 和 Tensorflow 2.0 神经网络分类表格数据")
url = 'https://raw.githubusercontent.com/wshuyi/demo-customer-churn-ann/master/customer_churn.csv'
filename = wget.detect_filename(url)
wget.download(url)

100% [........................................................] 684858 / 684858

'customer_churn.csv'

In [5]:
# 读取csv文件数据
import pandas as pd
df = pd.read_csv('customer_churn.csv')

In [6]:
#查看前几行数据
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [7]:
#查看有哪些列
df.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

##### 对列进行甄别
RowNumber：行号，这个对于模型没用，忽略
CustomerID：用户编号，这个是顺序发放的，忽略
Surname：用户姓名，对流失没有影响，忽略
CreditScore：信用分数，这个很重要，保留
Geography：用户所在国家/地区，这个有影响，保留
Gender：用户性别，可能有影响，保留
Age：年龄，影响很大，年轻人更容易切换银行，保留
Tenure：当了本银行多少年用户，很重要，保留
Balance：存贷款情况，很重要，保留
NumOfProducts：使用产品数量，很重要，保留
HasCrCard：是否有本行信用卡，很重要，保留
IsActiveMember：是否活跃用户，很重要，保留
EstimatedSalary：估计收入，很重要，保留
Exited：是否已流失，这将作为我们的标签数据

In [8]:
import warnings  
with warnings.catch_warnings():  
    warnings.filterwarnings("ignore",category=FutureWarning)
    import tensorflow as tf
    from tensorflow import keras
    from tensorflow.keras.preprocessing.text import Tokenizer

print('ready') 

ready


In [9]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from tensorflow import feature_column

设定一些随机种子值。这主要是为了保证结果可复现。\
首先是 Tensorflow 中的随机种子取值，设定为 1 。

In [10]:
tf.random.set_seed(1)

In [21]:
# 按照 80:20 的比例，把总体数据(df)分成训练集和测试集。
train, test = train_test_split(df, test_size=0.2, random_state=1)

In [22]:
#把现有训练集(train)的数据，按照 80:20 的比例，分成最终的训练集，以及验证集。
train, valid = train_test_split(train, test_size=0.2, random_state=1)

#### 特征工程（feature engineering）。
因为我们使用的是表格数据（tabular data），属于结构化数据。因此特征工程相对简单一些。


In [23]:
#先初始化一个空的特征列表。
feature_columns = []

In [24]:
#然后，我们指定，哪些列是数值型数据（numeric data）。
numeric_columns = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']

In [25]:
#对于这些列，只需要直接指定类型，加入咱们的特征列表就好。
for header in numeric_columns:
  feature_columns.append(feature_column.numeric_column(header))

#### 下面是比较讲究技巧的部分了，就是类别数据。
类别数据的特点，在于不能直接用数字描述。例如 Geography 包含了国家/地区名称。如果你把法国指定为1， 德国指定为2，电脑可能自作聪明，认为“德国”是“法国”的2倍.\
先看看都有哪些列：

In [26]:
categorical_columns = ['Geography', 'Gender', 'HasCrCard', 'IsActiveMember']

所以这里编了一个函数，把一个类别列名输入进去，让 Tensorflow 帮我们将其转换成它可以识别的类别形式。\
例如把法国按照 [0, 0, 1]，德国按照 [0, 1, 0] 来表示。这样就不会有数值意义上的歧义了。

In [27]:
def get_one_hot_from_categorical(colname):
  categorical = feature_column.categorical_column_with_vocabulary_list(colname, train[colname].unique().tolist())
  return feature_column.indicator_column(categorical)

In [28]:
#尝试输入 Geography 一项，测试一下函数工作是否正常。
geography = get_one_hot_from_categorical('Geography'); geography

IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='Geography', vocabulary_list=('France', 'Spain', 'Germany'), dtype=tf.string, default_value=-1, num_oov_buckets=0))

In [29]:
#下面我们放心大胆地把所有类别数据列都在函数里面跑一遍，并且把结果加入到特征列表中。
for col in categorical_columns:
  feature_columns.append(get_one_hot_from_categorical(col))

In [31]:
#看看此时的特征列表内容：6个数值类型，4个类别类型，都没问题了。
feature_columns

[NumericColumn(key='CreditScore', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='Age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='Tenure', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='Balance', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='NumOfProducts', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='EstimatedSalary', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='Geography', vocabulary_list=('France', 'Spain', 'Germany'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='Gender', vocabulary_list=('Male', 'Female'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(c

####  下面该构造模型了。

直接采用 Tensorflow 2.0鼓励开发者使用的 Keras 高级 API 来拼搭一个简单的深度神经网络模型。

In [32]:
from tensorflow.keras import layers
#把刚刚整理好的特征列表，利用 DenseFeatures 层来表示。
#把这样的一个初始层，作为模型的整体输入层。
feature_layer = layers.DenseFeatures(feature_columns); feature_layer

下面，我们顺序叠放两个中间层，分别包含200个，以及100个神经元。这两层的激活函数，我们都采用 relu 。
relu 函数大概长这个样子：
![title](v2_hd.jpg)

In [34]:
model = keras.Sequential([
  feature_layer,
  layers.Dense(200, activation='relu'),
  layers.Dense(100, activation='relu'),
  layers.Dense(1, activation='sigmoid')
])

我们希望输出结果是0或者1，所以这一层只需要1个神经元，而且采用的是 sigmoid 作为激活函数。\

sigmoid 函数的长相是这样的：
![title](v3_hd.jpg)

In [36]:
#模型搭建好了，下面我们指定3个重要参数，编译模型。
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

这里，我们选择优化器为 adam 。
![title](v4_b.webp)

因为评判二元分类效果，所以损失函数选的是 binary_crossentropy。

至于效果指标，我们使用的是准确率（accuracy）。

模型编译好之后。万事俱备，只差数据了。

你可能纳闷，一上来不就已经把训练、验证和测试集分好了吗？

没错，但那只是原始数据。我们模型需要接收的，是数据流。

在训练和验证过程中，数据都不是一次性灌入模型的。而是一批次一批次分别载入。每一个批次，称作一个 batch；相应地，批次大小，叫做 batch_size 。

为了方便咱们把 Pandas 数据框中的原始数据转换成数据流。我这里编写了一个函数。

In [37]:
def df_to_tfdata(df, shuffle=True, bs=32):
  df = df.copy()
  labels = df.pop('Exited')
  ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(df), seed=1)
  ds = ds.batch(bs)
  return ds

首先是把数据中的标记拆分出来。然后根据把数据读入到 ds 中。根据是否是训练集，我们指定要不要需要打乱数据顺序。然后，依据 batch_size 的大小，设定批次。这样，数据框就变成了神经网络模型喜闻乐见的数据流。

In [38]:
train_ds = df_to_tfdata(train)
valid_ds = df_to_tfdata(valid, shuffle=False)
test_ds = df_to_tfdata(test, shuffle=False)

这里，只有训练集打乱顺序。因为我们希望验证和测试集一直保持一致。只有这样，不同参数下，对比的结果才有显著意义。

有了模型架构，也有了数据，我们把训练集和验证集扔进去，让模型尝试拟合。这里指定了，跑5个完整轮次（epochs）。

In [39]:
model.fit(train_ds,
          validation_data=valid_ds,
          epochs=5)

Epoch 1/5
Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
200/200 [==============================] - 7s 33ms/step - loss: 5.7275 - accuracy: 0.4167 - val_loss: 3.1814 - val_accuracy: 0.7937
Epoch 2/5
200/200 [==============================] - 5s 24ms/step - loss: 3.1139 - accuracy: 0.7988 - val_loss: 3.1814 - val_accuracy: 0.7937
Epoch 3/5
200/200 [==============================] - 5s 24ms/step - loss: 3.1139 - accuracy: 0.7988 - val_loss: 3.1814 - val_accuracy: 0.7937
Epoch 4/5
200/200 [==============================] - 5s 24ms/step - loss: 3.1139 - accuracy: 0.7988 - val_loss: 3.1814 - val_accuracy: 0.7937
Epoch 5/5
200/200 [==============================] - 5s 24ms/step - loss: 3.1139 - a

你会看到，最终的验证集准确率接近80%。

In [41]:
#我们打印一下模型结构：
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_features (DenseFeature multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  3200      
_________________________________________________________________
dense_1 (Dense)              multiple                  20100     
_________________________________________________________________
dense_2 (Dense)              multiple                  101       
Total params: 23,401
Trainable params: 23,401
Non-trainable params: 0
_________________________________________________________________


In [42]:
#下面，我们把测试集放入模型中，看看模型效果如何。
model.evaluate(test_ds)

63/63 [==============================] - 1s 13ms/step - loss: 3.1829 - accuracy: 0.7925


[3.1829258591409713, 0.7925]